In [1]:
#ProgramName: Daily-06b-UpdateMasterDerivedChangingState
#Purpose: Weekly Update Master Catalog - Routine Maintenance
#Author:  Greg Turmel, Director, Data Governance 
#Date:    2020.08.30 - 2021.06.30
#Errata:  0.1 Improvements can be made to script using for/looping through the metadata captured

import os, sys, argparse, csv, pyodbc, sql, time, datetime
import sqlalchemy as db

from dotenv import load_dotenv # add this line
import pandas as pd
import numpy as np

now01 = datetime.datetime.now()

load_dotenv() # add this line
user = os.getenv('MySQLeUser')
password = os.getenv('MySQLeUserPass')
host = os.getenv('MySQLeHOST')
db = os.getenv('MySQLeDB')

In [ ]:
# This segment builds the appropriatte file system structure as a variable driven exercise
# Take time to set your 'eeeeeeeeee' number as variable 'pn' below 
# and set the program directory variable called 'programDirectory' before running
# ===================================================================================================================
pn = r'eeeeeeeeee'            #This represents the windows system employee login folder - IBM team uses a 9 number
# ===================================================================================================================

programDirectory = 'Daily' # Update this variable to wherever you want the program subfolder/files to be located 
un = r'C:\Users'
cn = r'Documents\Py'
an = r'Master\Process'
bn = r'Master\Reports'

sn = r'Master\sql'
tn = r'config'

#Set a parent directory
parentDirectory = "{}\{}\{}".format(un,pn,cn)
print('Parent Directory is: ', parentDirectory)
mode = 0o666

#Set path location for working with local file(s)
path = os.path.join(parentDirectory, programDirectory,)
pathMP = os.path.join(parentDirectory, programDirectory, an)
pathMR = os.path.join(parentDirectory, programDirectory, bn)
pathMS = os.path.join(parentDirectory, programDirectory, sn)
pathMT = os.path.join(parentDirectory, programDirectory, tn)

procpath = pathMP
csvpath = pathMR
sqlpath = pathMS
configpath = pathMT

try:
    if not os.path.exists(path):
        os.makedirs(path, mode)
        print('Program Directory subfolder has been created: ', programDirectory)
    else:
        print('Directory:', programDirectory, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMP):
        os.makedirs(pathMP, mode)
        print('Program Directory subfolder has been created: ', pathMP)
    else:
        print('Directory:', pathMP, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMR):
        os.makedirs(pathMR, mode)
        print('Program Directory subfolder has been created: ', pathMR)
    else:
        print('Directory:', pathMR, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMS):
        os.makedirs(pathMS, mode)
        print('Program Directory subfolder has been created: ', pathMS)
    else:
        print('Directory:', pathMS, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

try:
    if not os.path.exists(pathMT):
        os.makedirs(pathMT, mode)
        print('Program Directory subfolder has been created: ', pathMT)
    else:
        print('Central Config Directory:', pathMT, '>>>>> Note ---- this folder already exists <<<<<')
except OSError as error:
    print(error)
    pass

now01 = datetime.datetime.now()
print(': ', now01.strftime("%Y-%m-%d %H:%M:%S"))

##################################################################################################################

Note: This Notebook only updates the main Master
Validate tables from 6a collection activities and Master2T

##################################################################################################################

In [18]:
#This next series of notebook 'updates' the Master(Main) table 
now01 = datetime.datetime.now()
print('Begin: ', now01.strftime("%Y-%m-%d %H:%M:%S"))

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to zeroStatic

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC = 0) 
AND (S.AVGDD = 0) 
AND (S.SUMRC = 0) 
AND (S.SUMDD = 0) THEN
UPDATE SET [DerivedStaticChging]='zeroStatic';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master zeroStatic updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today ***


In [19]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0)
AND (S.AVGDD = 0) 
AND (S.SUMRC > 0) 
AND (S.SUMDD = 0) THEN
UPDATE SET [DerivedStaticChging]='Static';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master static updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today ***


In [20]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0 OR S.AVGRC < 0) 
AND (S.AVGDD > 0 OR S.AVGDD < 0) 
AND S.SUMRC > 0 
AND (S.SUMDD > 0 OR S.SUMDD < 0) THEN
UPDATE SET [DerivedStaticChging]='Chg-d';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master Chg-d status updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today ***


In [21]:
#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
MERGE [Greg].[dbo].[Master] T
USING dbo.[MasterPRODChangeTracking] S ON T.DatabaseName=S.DatabaseName AND T.SchemaName=S.SchemaName AND T.TableName=S.TableName
WHEN MATCHED 
AND (S.AVGRC > 0 AND S.AVGRC < 100) 
AND (S.AVGDD > 0 OR S.AVGDD < 0) 
AND (S.SUMRC > 0)
AND (S.SUMDD > 0 OR S.SUMDD < 100/3) THEN
UPDATE SET [DerivedStaticChging]='Chg-p';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master Chg-p(periodic) updated for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today ***


In [22]:
#ProgrammingError: ('42000', "[42000] 
#[Microsoft][ODBC SQL Server Driver][SQL Server]
#An action of type 'UPDATE' is not allowed in the 'WHEN NOT MATCHED' clause of a MERGE statement. 
#(10710) (SQLExecDirectW)")

#Solution: reverse the statement and match by source updating the specific target column with a value
    
    

#Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DEVODSSQL;'
                      'Database=Greg;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
#If Exists, Crear out any data from the tempGreg table if last used 
cursor.execute(''' 
SET TRANSACTION ISOLATION LEVEL READ UNCOMMITTED
SET NOCOUNT ON
SET ANSI_WARNINGS OFF
IF OBJECT_ID('tempdb..#tempGreg') IS NOT NULL 
BEGIN 
    DROP TABLE #tempGreg
END 
;
'''
,)
conn.commit()

#Update the Master table with any changes found to Static with data

cursor = conn.cursor()
cursor.execute(''' 
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'Assessment' AND SchemaName LIKE 'dbo' AND TableName LIKE 'ASSESS_COG_RAW_BKP';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_AllGrades';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_AwardsIssued';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_CLOE';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_EnrollmentsandWithdrawals';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted chg-d' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_FinalGrades';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_ImpersonatedSessionHistory';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_ads_InstructorUsage';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_bds_GradeSchemeRanges_temp';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'import_bds_QuizUserAnswerResponses_temp';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'TMP_GOC_A';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'd2l' AND TableName LIKE 'TMP_GOC_B';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted chg-p' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'EdCloud_Import_BM_Items';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_AD_DATA';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_STU_EMAIL_GSDR';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_STU_EMAIL_SYN';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'TMP_TBL';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'DownStreamFeeds' AND SchemaName LIKE 'dbo' AND TableName LIKE 'tmp_test_tbl';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted zeroStatic' WHERE DatabaseName LIKE 'GCPS_Operations' AND SchemaName LIKE 'ERROR' AND TableName LIKE ' STUDENT_ABS';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'GSDR' AND SchemaName LIKE 'dbo'  AND TableName LIKE 'kstemp1';
UPDATE [Greg].[dbo].[Master] SET [DerivedStaticChging]='Deleted static' WHERE DatabaseName LIKE 'GSDR' AND SchemaName LIKE 'dbo'  AND TableName LIKE 'srfile';
'''
,)
conn.commit()

conn.close()
now01 = datetime.datetime.now()
print(' *** Note:Greg.dbo.Master updated deleted status for today ***', now01.strftime("%Y-%m-%d %H:%M:%S"))

 *** Note:  database table Greg.dbo.Master change status updated for today ***
